In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
import warnings
warnings.filterwarnings("ignore")

#### Importing and cleaning the 2019 file

In [3]:
#Import file
event_details=pd.read_csv("Resources/StormEvents_details-ftp_v1.0_d2019_c20190617.csv", encoding="UTF-8")

#Parse year and month description
event_details['BEGIN_YEARMONTH']=event_details['BEGIN_YEARMONTH'].astype(str)
event_details['BEGIN_YEAR']=event_details['BEGIN_YEARMONTH'].str[0:4]
event_details['BEGIN_MONTH']=event_details['BEGIN_YEARMONTH'].str[4:6]

#Move the new columns to the first and second column
cols = list(event_details.columns)
cols = [cols[-1]] + cols[:-1]
event_details = event_details[cols]

cols = list(event_details.columns)
cols = [cols[-1]] + cols[:-1]
event_details = event_details[cols]

event_details.head()

,BEGIN_YEAR,BEGIN_MONTH,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,2019,01,201901,29,1200,201901,29,1900,133946,801726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong surface cold front crossed the region...,Snowfall amounts of 1 to 2 inches were observe...,CSV
1,2019,03,201903,9,810,201903,9,810,134106,814097,...,0.0,N,CADDO GAP,34.4,-93.62,34.4,-93.62,A strong storm system brought severe weather t...,A tractor shed was destroyed in Caddo Gap.,CSV
2,2019,03,201903,9,750,201903,9,750,134106,814096,...,1.0,NNE,SUBIACO,35.3,-93.64,35.3,-93.64,A strong storm system brought severe weather t...,Tree limbs were broken and shingles were blown...,CSV
3,2019,01,201901,12,1000,201901,13,1400,134278,804158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure tracked south of the region throu...,A report from Delaware showed that 4.5 inches ...,CSV
4,2019,01,201901,12,1000,201901,13,1400,134278,804159,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure tracked south of the region throu...,"Based on surrounding observations, it is estim...",CSV


In [4]:
#Drop columns that are not needed
event_details.drop(columns={'BEGIN_YEARMONTH','CZ_TYPE','CZ_FIPS','CZ_NAME','CZ_TIMEZONE','MAGNITUDE','MAGNITUDE_TYPE',
                    'CATEGORY','TOR_F_SCALE','TOR_LENGTH','TOR_WIDTH','TOR_OTHER_WFO','TOR_OTHER_CZ_STATE',
                    'TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','BEGIN_AZIMUTH','BEGIN_RANGE',
                    'END_RANGE','END_AZIMUTH','END_LOCATION','EPISODE_NARRATIVE','EVENT_NARRATIVE',
                    'DATA_SOURCE','CZ_TIMEZONE','BEGIN_DATE_TIME','END_DATE_TIME'}, inplace=True)

In [5]:
#filter and keep only flood events
event_details = event_details.loc[(event_details['EVENT_TYPE'] == 'Flood') | (event_details['EVENT_TYPE'] == 'Flash Flood')]

event_details.reset_index(inplace=True)
event_details.drop(columns={'index'},inplace=True)
event_details.head()

,BEGIN_YEAR,BEGIN_MONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,...,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,FLOOD_CAUSE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
0,2019,02,14,1610,201902,14,1905,133682,800014,HAWAII,...,0,0.00K,0.00K,Emergency Manager,Heavy Rain,HAIKU,20.9177,-156.3282,20.9220,-156.3141
1,2019,02,23,1600,201902,23,1902,133684,800018,HAWAII,...,0,0.00K,0.00K,Emergency Manager,Heavy Rain,KURTISTOWN,19.5721,-155.0290,19.5738,-155.0273
2,2019,01,24,1300,201901,24,1500,135012,809140,VIRGINIA,...,0,0.00K,0.00K,Law Enforcement,Heavy Rain,TREMONT,38.8459,-77.2383,38.8467,-77.2375
3,2019,01,24,1300,201901,24,1500,135012,809141,VIRGINIA,...,0,0.00K,0.00K,Law Enforcement,Heavy Rain,VALE,38.9108,-77.3630,38.9119,-77.3642
4,2019,01,24,1300,201901,24,1500,135012,809142,VIRGINIA,...,0,0.00K,0.00K,Law Enforcement,Heavy Rain,CLARKS CROSSING,38.9182,-77.3015,38.9159,-77.2967


In [6]:
#save clean data 
event_details.to_csv("Output/Flood Events 2019.csv", index=False)

In [7]:
# Create Engine
rds_connection_string =('pg8000://{username}:{password}@{ipaddress}:{port}/{dbname}'
               .format(username="postgres",
                       password="postgres",
                       ipaddress="localhost",
                       port=5432,
                       dbname="Storm")
              )
engine=create_engine(f'postgresql+{rds_connection_string}')

In [8]:
event_details.to_sql(name='Flood_Events_2019', con=engine, if_exists='replace', index=False)

### Importing and cleaning the historical file

In [9]:
#Import glob loop through 20 years of storm data
import glob

path = "Resources" 
all_files = glob.glob(path + "/*.csv")
li=[]

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Create a dataframe using the imported data
storm_data = pd.concat(li, axis=0, ignore_index=True)

storm_data.head()

,AZIMUTH,BEGIN_AZIMUTH,BEGIN_DATE_TIME,BEGIN_DAY,BEGIN_LAT,BEGIN_LOCATION,BEGIN_LON,BEGIN_RANGE,BEGIN_TIME,BEGIN_YEARMONTH,...,TOR_F_SCALE,TOR_LENGTH,TOR_OTHER_CZ_FIPS,TOR_OTHER_CZ_NAME,TOR_OTHER_CZ_STATE,TOR_OTHER_WFO,TOR_WIDTH,WFO,YEAR,YEARMONTH
0,NaN,NaN,19-APR-65 13:50:00,19.0,30.58,NaN,-88.08,0.0,1350.0,196504.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,1965.0,NaN
1,NaN,NaN,25-JAN-65 21:50:00,25.0,38.78,NaN,-92.27,0.0,2150.0,196501.0,...,F0,0.1,NaN,NaN,NaN,NaN,10.0,NaN,1965.0,NaN
2,NaN,NaN,12-APR-65 04:27:00,12.0,40.70,NaN,-72.68,0.0,427.0,196504.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,1965.0,NaN
3,NaN,NaN,01-AUG-65 13:20:00,1.0,39.05,NaN,-76.05,0.0,1320.0,196508.0,...,F1,1.3,NaN,NaN,NaN,NaN,200.0,NaN,1965.0,NaN
4,NaN,NaN,04-SEP-65 19:40:00,4.0,43.02,NaN,-86.10,0.0,1940.0,196509.0,...,F2,4.1,NaN,NaN,NaN,NaN,33.0,NaN,1965.0,NaN


In [10]:
#Parse year and month description
storm_data['BEGIN_YEARMONTH']=storm_data['BEGIN_YEARMONTH'].astype(str)
storm_data['BEGIN_YEAR']=storm_data['BEGIN_YEARMONTH'].str[0:4]
storm_data['BEGIN_MONTH']=storm_data['BEGIN_YEARMONTH'].str[4:6]

#Move the new columns to the first and second column
cols = list(storm_data.columns)
cols = [cols[-1]] + cols[:-1]
storm_data = storm_data[cols]

cols = list(storm_data.columns)
cols = [cols[-1]] + cols[:-1]
storm_data = storm_data[cols]



In [11]:
#Drop columns that are not needed
storm_data.drop(columns={'BEGIN_YEARMONTH','END_YEARMONTH','END_DAY','END_TIME','CZ_TYPE','CZ_FIPS','CZ_NAME','CZ_TIMEZONE','MAGNITUDE','MAGNITUDE_TYPE',
                    'CATEGORY','TOR_F_SCALE','TOR_LENGTH','TOR_WIDTH','TOR_OTHER_WFO','TOR_OTHER_CZ_STATE',
                    'TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','BEGIN_AZIMUTH','BEGIN_RANGE',
                    'END_RANGE','END_AZIMUTH','END_LOCATION','EPISODE_NARRATIVE','EVENT_NARRATIVE',
                    'DATA_SOURCE','CZ_TIMEZONE','BEGIN_DATE_TIME','END_DATE_TIME'}, inplace=True)

In [12]:
#Retrive the total for damage and death
storm_data[["DEATHS_DIRECT",'DEATHS_INDIRECT','INJURIES_DIRECT','INJURIES_INDIRECT',"DAMAGE_PROPERTY","DAMAGE_CROPS"]].fillna(0, inplace=True)
storm_data["DEATH_TOLL"]=storm_data['DEATHS_DIRECT']+storm_data['DEATHS_INDIRECT']
storm_data['TOTAL_INJURIES']=storm_data['INJURIES_DIRECT']+storm_data['INJURIES_INDIRECT']

#change property and crop damage to numbers and add 1000
storm_data["DAMAGE_PROPERTY"]=storm_data['DAMAGE_PROPERTY'].str[:-1]
storm_data["DAMAGE_CROPS"]=storm_data['DAMAGE_CROPS'].str[:-1]

storm_data["DAMAGE_PROPERTY"]=(storm_data['DAMAGE_PROPERTY'].apply(pd.to_numeric))/1000
storm_data["DAMAGE_CROPS"]=(storm_data['DAMAGE_CROPS'].apply(pd.to_numeric))/1000

#storm_data.to_csv("Output/StormData.csv", index=False)

In [14]:
sum_groupby=storm_data[["EVENT_TYPE","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']]
sum_groupby["EVENT_TYPE"]=sum_groupby["EVENT_TYPE"].replace({"Flash Flood":"Flood","Excessive Heat":'Heat' })
sum_groupby=sum_groupby.groupby(["EVENT_TYPE"]).sum()

count_groupby=storm_data[["EVENT_TYPE","DAMAGE_PROPERTY"]]
count_groupby["EVENT_TYPE"]=count_groupby["EVENT_TYPE"].replace({"Flash Flood":"Flood"})
count_groupby=count_groupby.groupby(["EVENT_TYPE"]).count()
count_groupby.rename(columns={'DAMAGE_PROPERTY':'COUNT'}, inplace=True)

storm_summary=pd.merge(count_groupby,sum_groupby, on="EVENT_TYPE")

storm_summary.sort_values("COUNT", ascending=False).head()

,COUNT,DAMAGE_PROPERTY,DAMAGE_CROPS,DEATH_TOLL,TOTAL_INJURIES
EVENT_TYPE,,,,,
Thunderstorm Wind,202120,2673.07473,171.12315,555.0,5040.0
Hail,122779,747.71160,493.16862,12.0,766.0
Flood,91109,2825.14544,396.51003,1742.0,1786.0
High Wind,40554,723.93901,28.41502,359.0,1618.0
Winter Weather,39158,61.96100,0.01600,818.0,5298.0


In [15]:
storm_summary.sort_values("DAMAGE_PROPERTY", ascending=False).head()

,COUNT,DAMAGE_PROPERTY,DAMAGE_CROPS,DEATH_TOLL,TOTAL_INJURIES
EVENT_TYPE,,,,,
Flood,91109,2825.14544,396.51003,1742.0,1786.0
Thunderstorm Wind,202120,2673.07473,171.12315,555.0,5040.0
Tornado,21096,1428.95249,101.19155,1880.0,26115.0
Hail,122779,747.71160,493.16862,12.0,766.0
High Wind,40554,723.93901,28.41502,359.0,1618.0


In [16]:
storm_summary.sort_values("TOTAL_INJURIES", ascending=False).head()

,COUNT,DAMAGE_PROPERTY,DAMAGE_CROPS,DEATH_TOLL,TOTAL_INJURIES
EVENT_TYPE,,,,,
Tornado,21096,1428.95249,101.19155,1880.0,26115.0
Heat,6804,12.49400,9.34990,2886.0,14853.0
Winter Weather,39158,61.96100,0.01600,818.0,5298.0
Thunderstorm Wind,202120,2673.07473,171.12315,555.0,5040.0
Lightning,10858,544.58983,1.97564,740.0,4387.0


In [17]:
storm_summary.sort_values("DEATH_TOLL", ascending=False).head()

,COUNT,DAMAGE_PROPERTY,DAMAGE_CROPS,DEATH_TOLL,TOTAL_INJURIES
EVENT_TYPE,,,,,
Heat,6804,12.49400,9.34990,2886.0,14853.0
Tornado,21096,1428.95249,101.19155,1880.0,26115.0
Flood,91109,2825.14544,396.51003,1742.0,1786.0
Hurricane (Typhoon),816,86.94492,30.01062,1099.0,3747.0
Rip Current,752,0.16300,0.00000,865.0,661.0


In [18]:
#Sumarize flood data
flood_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Flood') | (storm_data['EVENT_TYPE'] == 'Flash Flood')]

flood_data.reset_index(inplace=True)
flood_data.drop(columns={'index'},inplace=True)
flood_data.head()

flood_data.to_csv("Output/FloodData.csv", index=True)

sum_groupby=flood_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']]
sum_groupby=sum_groupby.groupby(["BEGIN_YEAR"]).sum()
count_groupby=flood_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'FLOOD_COUNT'}, inplace=True)
count_groupby

flood_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
flood_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)
flood_summary

flood_summary.to_csv("Output/FloodSummary.csv", index=True)

In [ ]:
#Sumarize Thunderstorm data
thunderstorm_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Thunderstorm Wind')]

thunderstorm_data.reset_index(inplace=True)
thunderstorm_data.drop(columns={'index'},inplace=True)

thunderstorm_data.to_csv("Output/ThunderstormData.csv", index=True)

sum_groupby=thunderstorm_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']]
sum_groupby=sum_groupby.groupby(["BEGIN_YEAR"]).sum()
count_groupby=thunderstorm_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'FLOOD_COUNT'}, inplace=True)


thunderstorm_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
thunderstorm_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)

thunderstorm_summary.to_csv("Output/ThunderstormSummary.csv", index=True)

In [ ]:
#Sumarize hail data
hail_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Hail')]

hail_data.reset_index(inplace=True)
hail_data.drop(columns={'index'},inplace=True)

hail_data.to_csv("Output/HailData.csv", index=True)

sum_groupby=hail_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=hail_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'HAIL_COUNT'}, inplace=True)

hail_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
hail_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)


hail_summary.to_csv("Output/HailSummary.csv", index=True)

In [ ]:
#Sumarize high wind data
highwind_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'High Wind')]

highwind_data.reset_index(inplace=True)
highwind_data.drop(columns={'index'},inplace=True)

highwind_data.to_csv("Output/HighWindData.csv", index=True)

sum_groupby=highwind_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=highwind_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'HIGHWIND_COUNT'}, inplace=True)

highwind_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
highwind_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)


hail_summary.to_csv("Output/HighWindSummary.csv", index=True)

In [ ]:
#Sumarize winter weather data
winterweather_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Winter Weather')]

winterweather_data.reset_index(inplace=True)
winterweather_data.drop(columns={'index'},inplace=True)

winterweather_data.to_csv("Output/WinterWeatherData.csv", index=True)

sum_groupby=winterweather_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=winterweather_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'WINTERWEATHER_COUNT'}, inplace=True)

winterweather_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
winterweather_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)


winterweather_summary.to_csv("Output/WinterWeatherSummary.csv", index=True)

In [ ]:
#Sumarize Tornado data
tornado_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Tornado')]

tornado_data.reset_index(inplace=True)
tornado_data.drop(columns={'index'},inplace=True)

tornado_data.to_csv("Output/TornadoData.csv", index=True)

sum_groupby=tornado_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=tornado_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'TORNADO_COUNT'}, inplace=True)

tornado_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
tornado_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)

tornado_summary.to_csv("Output/TornadoSummary.csv", index=True)

In [ ]:
#Sumarize Heat data
heat_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Heat')]

heat_data.reset_index(inplace=True)
heat_data.drop(columns={'index'},inplace=True)

heat_data.to_csv("Output/HeatData.csv", index=True)

sum_groupby=heat_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=heat_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'HEAT_COUNT'}, inplace=True)

heat_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
heat_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)

heat_summary.to_csv("Output/HeatSummary.csv", index=True)

In [ ]:
#Sumarize Heat data
heat_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Heat')]

heat_data.reset_index(inplace=True)
heat_data.drop(columns={'index'},inplace=True)

heat_data.to_csv("Output/HeatData.csv", index=True)

sum_groupby=heat_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=heat_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'HEAT_COUNT'}, inplace=True)

heat_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
heat_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)

heat_summary.to_csv("Output/HeatSummary.csv", index=True)

In [ ]:
#Sumarize Lightning data
lightning_data= storm_data.loc[(storm_data['EVENT_TYPE'] == 'Lightning')]

lightning_data.reset_index(inplace=True)
lightning_data.drop(columns={'index'},inplace=True)

lightning_data.to_csv("Output/LightningData.csv", index=True)

sum_groupby=lightning_data[["BEGIN_YEAR","DAMAGE_PROPERTY","DAMAGE_CROPS","DEATH_TOLL",'TOTAL_INJURIES']].groupby(["BEGIN_YEAR"]).sum()
count_groupby=lightning_data[["BEGIN_YEAR","DAMAGE_PROPERTY"]].groupby(["BEGIN_YEAR"]).count()

count_groupby.rename(columns={'DAMAGE_PROPERTY':'Lightning_COUNT'}, inplace=True)

lightning_summary=pd.merge(count_groupby,sum_groupby, on="BEGIN_YEAR")
lightning_summary.rename(index={"BEGIN_YEAR":"YEAR"}, inplace=True)

lightning_summary.to_csv("Output/LightningSummary.csv", index=True)